# Gender Bias in Autocomplete AI

The AI at hand is to predict next words based on what users have typed. It's pretty much how autocomplete in Google search works. Let's try to make the autocomplete algorithm less biased toward a certain gender.

The code learns from an existing corpora (text-based dataset), and performs autocomplete when receiving a word input by a user.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import pandas as pd
import collections

## Training with Bias

In [ ]:
def preprocess(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
    return tokens

dataset = pd.read_csv("/content/drive/My Drive/Autocomplete/Autocomplete Dataset.csv")
dataset["Comments"] = dataset["Comments"].str.replace('\r\n', '')
text_list = dataset["Comments"].tolist()
text = ' '.join(text_list)

# Preprocess the text
tokens = preprocess(text)

# larger range
def train_model(tokens):
    model = collections.defaultdict(list)
    for i in range(len(tokens)-1):
        key = tokens[i]
        values = tokens[i-2:i] + tokens[i+1:i+3]
        model[key].extend(values)
    return model

# Train the model
model = train_model(tokens)

import random

def generate_prediction(model, prefix):
    if prefix in model:
        suffixes = model[prefix]
        return random.choice(suffixes)
    else:
        return None

def check_adjective(word):
    tagged_word = nltk.pos_tag([word])
    pos = tagged_word[0][1]
    return pos.startswith('JJ')

# Take input from user
input_str = input("Enter word: ")
output_num = input("How many words do you want to generate: ")

# Preprocess the input
input_tokens = preprocess(input_str)

# Generate prediction
count = 0
while count < int(output_num):
    new_word = generate_prediction(model, input_tokens[-1])
    if new_word != "women" and new_word != "men" and check_adjective(new_word):
      count += 1
      input_tokens.append(new_word)

# Print the prediction
if input_tokens:
    print("Next word prediction:", input_tokens[1:])
else:
    print("No prediction found.")

Enter word: women
How many words do you want to generate: 5
Next word prediction: ['empathetic', 'sensitive', 'empathetic', 'sensitive', 'empathetic']


### Improve the Model

Let's inspect the transition matrix of our model and modify the model to make the probability of predicting biased words smaller.

In [ ]:
import sklearn
from collections import Counter

In [ ]:
def create_transition_matrix(model):
  for key, entry in model.items():
    model[key] = Counter(entry)
  transition_matrix = pd.DataFrame(model).fillna(0)
  tm_normalized = transition_matrix.div(transition_matrix.sum(axis=1), axis=0)
  return tm_normalized

In [ ]:
transition_matrix = create_transition_matrix(model)

In [ ]:
transition_probability_for_women = {k: v[0] for k, v in sorted(transition_matrix.loc[["women"]].to_dict(orient='list').items(), key=lambda item: item[1], reverse=True)}

In [ ]:
list(transition_probability_for_women.items())[:10]

[('men', 0.030927835051546393),
 ('better', 0.030927835051546393),
 ('prioritize', 0.030927835051546393),
 ('inclined', 0.030927835051546393),
 ('towards', 0.030927835051546393),
 ('likely', 0.030927835051546393),
 ('caring', 0.020618556701030927),
 ('emotional', 0.020618556701030927),
 ('multitasking', 0.020618556701030927),
 ('natural', 0.020618556701030927)]

Now you can try manually changing the probability of some biased words to make it less likely to appear in the autocomplete results when you input "women". However, make the sure the transition probability to all the words sum up to 1.

In [ ]:
biased_word = [input("What words do you think are biased? " )]

What words do you think are biased?emotional, caring


In [ ]:
penalty_factor = float(input("How much do you want to discount the biased words? Please enter a number between 0 and 1 " ))

How much do you want to discount the biased words? Please enter a number between 0 and 1  0.5


In [ ]:
def bias_fine_tune(biased_word,penalty_factor, transition_probability, key):
  for key, prob in transition_probability.items():
    if key in biased_word:
      transition_probability[key] *= penalty_factor
  transition_matrix.loc[key] = transition_probability
  transition_matrix_normalized = transition_matrix.div(transition_matrix.sum(axis=1), axis=0)
  return transition_matrix_normalized

In [ ]:
debiased_transition_matrix = bias_fine_tune(biased_word,penalty_factor,transition_probability_for_women, "women");

Now we can use the new transition matrix to make the predictions. Let's see if there's any difference in the results.

In [ ]:
def generate_prediction_with_transition_matrix(model, prefix, number):
    count = 0
    predicted_words = []
    if prefix in transition_matrix.index:
      while count < int(number):
        possible_words = list(transition_matrix.columns)
        probabilities = list(transition_matrix.loc[prefix])
        new_word = random.choices(possible_words, weights=probabilities, k=1)[0]
        if new_word != "women" and new_word != "men" and check_adjective(new_word):
          count += 1
          predicted_words.append(new_word)
      return predicted_words
    else:
        return None

In [ ]:
input_str = input("Enter word: ")
output_num = input("How many words do you want to generate: ")
generate_prediction_with_transition_matrix(debiased_transition_matrix,input_str,output_num)

Enter word: women
How many words do you want to generate: 5


['inclined', 'empathetic', 'emotional', 'empathetic', 'organizational']

Now do the same thing for the word transition probability for men.

In [ ]:
transition_probability_for_men = {k: v[0] for k, v in sorted(transition_matrix.loc[["men"]].to_dict(orient='list').items(), key=lambda item: item[1], reverse=True)}
list(transition_probability_for_men.items())[:10]

[('less', 0.07142857142857142),
 ('emotional', 0.03571428571428571),
 ('inclined', 0.03571428571428571),
 ('women', 0.026785714285714284),
 ('nurturing', 0.026785714285714284),
 ('confident', 0.026785714285714284),
 ('focused', 0.026785714285714284),
 ('interested', 0.026785714285714284),
 ('natural', 0.017857142857142856),
 ('assertive', 0.017857142857142856)]

In [ ]:
biased_word = [input("What words do you think are biased? " )]
penalty_factor = float(input("How much do you want to discount the biased words? Please enter a number between 0 and 1 " ))

What words do you think are biased? confident
How much do you want to discount the biased words? Please enter a number between 0 and 1 0.5


In [ ]:
debiased_transition_matrix = bias_fine_tune(biased_word,penalty_factor,transition_probability_for_men, "men");

In [ ]:
input_str = input("Enter word: ")
output_num = input("How many words do you want to generate: ")
generate_prediction_with_transition_matrix(debiased_transition_matrix,input_str,output_num)

Enter word: men
How many words do you want to generate: 5


['emotional', 'inclined', 'potential', 'interpersonal', 'potential']